In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))


In [3]:
import pandas as pd
from src.config import TRANSFORMED_DATA_DIR

# Load 28-day window Citi Bike features
df = pd.read_parquet(TRANSFORMED_DATA_DIR / "citibike_tabular_data_28d.parquet")


In [4]:
from datetime import datetime
from src.data_utils import split_time_series_data

X_train, y_train, X_test, y_test = split_time_series_data(
    df,  # or df_ts or your actual DataFrame
    cutoff_date=datetime(2025, 1, 1, 0, 0, 0),  # within your date range
    target_column="target"  # actual target column name
)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)


(1665, 674)
(1665,)
(600, 674)
(600,)


In [5]:
past_ride_columns = [c for c in X_train.columns if c.startswith("rides_")]
X_train_only_numeric = X_train[past_ride_columns]
X_test_only_numeric = X_test[past_ride_columns]


In [6]:
def average_rides_last_4_weeks(X: pd.DataFrame) -> pd.DataFrame:
    last_4_weeks_columns = [
        f"rides_t-{7*24}",  # 1 week ago
        f"rides_t-{14*24}", # 2 weeks ago
        f"rides_t-{21*24}", # 3 weeks ago
        f"rides_t-{28*24}"  # 4 weeks ago
    ]

    for col in last_4_weeks_columns:
        if col not in X.columns:
            raise ValueError(f"Missing required column: {col}")

    X["average_rides_last_4_weeks"] = X[last_4_weeks_columns].mean(axis=1)
    return X


In [7]:
from sklearn.preprocessing import FunctionTransformer

add_feature_average_rides_last_4_weeks = FunctionTransformer(
    average_rides_last_4_weeks, validate=False
)
add_feature_average_rides_last_4_weeks.fit_transform(X_train)


,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id,average_rides_last_4_weeks
0,0,0,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,2024-02-03,HB101,0.00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-02-04,HB101,0.00
2,0,0,0,0,0,0,1,6,5,3,...,0,0,0,0,0,0,0,2024-02-05,HB101,0.00
3,4,5,4,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,2024-02-06,HB101,1.00
4,0,0,0,0,0,0,0,3,9,2,...,4,7,7,3,5,3,1,2024-02-07,HB101,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-12-27,JC115,0.25
2141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-12-28,JC115,0.00
2142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-12-29,JC115,0.75
2143,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-12-30,JC115,0.75


In [8]:
from sklearn.base import BaseEstimator, TransformerMixin

class TemporalFeatureEngineer(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        X_ = X.copy()
        X_["hour"] = X_["pickup_hour"].dt.hour
        X_["day_of_week"] = X_["pickup_hour"].dt.dayofweek
        return X_.drop(columns=["pickup_hour", "pickup_location_id"])

add_temporal_features = TemporalFeatureEngineer()
add_temporal_features.fit_transform(X_train)


,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,average_rides_last_4_weeks,hour,day_of_week
0,0,0,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0.00,0,5
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.00,0,6
2,0,0,0,0,0,0,1,6,5,3,...,0,0,0,0,0,0,0,0.00,0,0
3,4,5,4,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,1.00,0,1
4,0,0,0,0,0,0,0,3,9,2,...,4,7,7,3,5,3,1,0.00,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.25,0,4
2141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.00,0,5
2142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.75,0,6
2143,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.75,0,0


In [9]:
import lightgbm as lgb
from sklearn.pipeline import make_pipeline

pipeline = make_pipeline(
    add_feature_average_rides_last_4_weeks,
    add_temporal_features,
    lgb.LGBMRegressor()
)

pipeline.fit(X_train, y_train)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.021231 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13403
[LightGBM] [Info] Number of data points in the train set: 1665, number of used features: 674
[LightGBM] [Info] Start training from score 0.592192


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x000002E3063D1080>)),
                ('temporalfeatureengineer', TemporalFeatureEngineer()),
                ('lgbmregressor', LGBMRegressor())])

In [10]:
add_temporal_features = TemporalFeatureEngineer()
add_temporal_features.fit_transform(X_train)

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,average_rides_last_4_weeks,hour,day_of_week
0,0,0,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,0.00,0,5
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.00,0,6
2,0,0,0,0,0,0,1,6,5,3,...,0,0,0,0,0,0,0,0.00,0,0
3,4,5,4,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,1.00,0,1
4,0,0,0,0,0,0,0,3,9,2,...,4,7,7,3,5,3,1,0.00,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.25,0,4
2141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.00,0,5
2142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.75,0,6
2143,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0.75,0,0


In [11]:
from sklearn.metrics import mean_absolute_error

predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae:.4f}")


0.9040


In [12]:
X_train

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_hour,pickup_location_id,average_rides_last_4_weeks
0,0,0,0,0,0,0,0,0,1,2,...,0,0,0,0,0,0,0,2024-02-03,HB101,0.00
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-02-04,HB101,0.00
2,0,0,0,0,0,0,1,6,5,3,...,0,0,0,0,0,0,0,2024-02-05,HB101,0.00
3,4,5,4,0,0,0,0,0,0,3,...,0,0,0,0,0,0,0,2024-02-06,HB101,1.00
4,0,0,0,0,0,0,0,3,9,2,...,4,7,7,3,5,3,1,2024-02-07,HB101,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2140,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-12-27,JC115,0.25
2141,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-12-28,JC115,0.00
2142,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-12-29,JC115,0.75
2143,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2024-12-30,JC115,0.75


In [13]:
from src.experiment_utils import set_mlflow_tracking, log_model_to_mlflow
from dotenv import load_dotenv

load_dotenv()
mlflow = set_mlflow_tracking()

log_model_to_mlflow(pipeline, X_test, "LGBMRegressorWFE-Citibike", "mean_absolute_error", score=test_mae)


INFO:src.experiment_utils:MLflow tracking URI and credentials set.
2025/05/10 05:06:14 INFO mlflow.tracking.fluent: Experiment with name 'LGBMRegressorWFE-Citibike' does not exist. Creating a new experiment.
INFO:src.experiment_utils:Experiment set to: LGBMRegressorWFE-Citibike
INFO:src.experiment_utils:Logged mean_absolute_error: 0.9039818760016904
c:\Users\Yaseen\anaconda3\envs\venv\Lib\site-packages\mlflow\types\utils.py:452: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https

2025/05/10 05:06:30 INFO mlflow.models.model: Found the following environment variables used during model inference: [HOPSWORKS_API_KEY]. Please check if you need to set them when deploying the model. To disable this message, set environment variable `MLFLOW_RECORD_ENV_VARS_IN_MODEL_LOGGING` to `false`.
Successfully registered model 'Pipeline'.
2025/05/10 05:06:37 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Pipeline, version 1
Created version '1' of model 'Pipeline'.
INFO:src.experiment_utils:Model logged with name: Pipeline


🏃 View run zealous-stag-405 at: https://dagshub.com/yaseensiddiqui36/cda500_final_project_citi_bike.mlflow/#/experiments/5/runs/67e48f5b93624a2b865581d56c31c262
🧪 View experiment at: https://dagshub.com/yaseensiddiqui36/cda500_final_project_citi_bike.mlflow/#/experiments/5
